In [ ]:
# ! pip install evaluate
# ! pip install bert_score
# ! pip install sacrebleu
# ! pip install nltk
# ! pip install rouge_score
# ! pip install torchmetrics
# # for BLEURT refer to https://github.com/google-research/bleurt 

# Load annotations

In [1]:
import json
import pprint 
import numpy as np
from evaluate import load

pp = pprint.PrettyPrinter(indent=4)
img_prompt_respond_file = '../data/memecap/filtered_meme_configs_5_attributes_memecap.json'
# Load predictions
predictions = []
with open(img_prompt_respond_file, 'r', encoding='utf-8') as json_file:
    predictions = json.load(json_file)
len(predictions), predictions[0]['meaning of the meme'], predictions[0]['image_dir'].split('/')[-1]

/home/shiling/anaconda3/envs/llava/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-06 10:14:09.242261: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-06 10:14:09.250734: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-06 10:14:09.261284: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-06 10:14

(464,
 "- The meme humorously suggests that a wife, after losing her memory, fell in love with and decided to remarry her own husband, indicating his unchanging worth and likeability as a partner, paralleled by the character's proud declaration of worthiness regarding his hammer.",
 'memes_d079np.png')

# Load gtound truth 

In [2]:
gt = []
with open('../data/memecap/meme-cap-main/data/memes-test.json', 'r', encoding='utf-8') as json_file:
    gt = json.load(json_file)
len(gt), gt[0]['meme_captions'], gt[0]['img_fname']
# concatenate all the references from the ground truth.
memecap_predictions_concatenated = []
memecap_processed_gt_concatenated = []
# copy the predictions to match the number of references
memecap_predictions_extended = []
memecap_processed_gt_extended = []
# Do nothing. One-prediction to multi-reference
memecap_predictions_multi_ref = []
memecap_processed_gt_multi_ref = []

for idx, meme_conf in enumerate(predictions):
    for meme in gt:
        if meme_conf['image_dir'].split('/')[-1] == meme['img_fname']:
            memecap_predictions_concatenated.append(meme_conf['meaning of the meme'])
            memecap_processed_gt_concatenated.append(''.join(meme['meme_captions']))
            
            memecap_predictions_extended += [meme_conf['meaning of the meme']] * len(meme['meme_captions'])
            memecap_processed_gt_extended += meme['meme_captions']

            memecap_predictions_multi_ref.append(meme_conf['meaning of the meme'])
            memecap_processed_gt_multi_ref.append(meme['meme_captions'])
len(memecap_processed_gt_concatenated), len(memecap_processed_gt_extended), len(memecap_predictions_extended), len(memecap_predictions_multi_ref), len(memecap_processed_gt_multi_ref)

(464, 1567, 1567, 464, 464)

# BLEURT
## Github repo
https://github.com/google-research/bleurt

In [5]:
def calculate_the_best_matches(target_scores, sample_size, multi_ref): 
    # Calculate the score for the best matches 
    restored_scores = []
    last_idx = 0
    for i in range(sample_size):
        restored_scores.append(target_scores[last_idx : last_idx + len(multi_ref[i])])
        last_idx += len(multi_ref[i]) 
    # print(restored_scores)
    maximized = [ max(ite) for ite in restored_scores]
    print('The average score for the best matches:', np.mean(maximized))

In [4]:
from bleurt import score

checkpoint = "/home/shiling/git/bleurt/BLEURT-20"
# references = ["This is a test.", "This is a test."]
# candidates = ["This is a test.", "This is the test."]
# [0.9881654977798462, 0.7926645278930664]

scorer = score.BleurtScorer(checkpoint)
# Extended predictions
scores = scorer.score(references=memecap_processed_gt_extended, candidates=memecap_predictions_extended)
print('Extended version:', np.mean(scores))

calculate_the_best_matches(scores, len(memecap_predictions_multi_ref), memecap_processed_gt_multi_ref)

# Concatenated references
scores = scorer.score(references=memecap_processed_gt_concatenated, candidates=memecap_predictions_concatenated)
print('Concatenated version:', np.mean(scores))


INFO:tensorflow:Reading checkpoint /home/shiling/git/bleurt/BLEURT-20.


INFO:tensorflow:Reading checkpoint /home/shiling/git/bleurt/BLEURT-20.


INFO:tensorflow:Config file found, reading.


INFO:tensorflow:Config file found, reading.


INFO:tensorflow:Will load checkpoint BLEURT-20


...

INFO:tensorflow:Loading model.


INFO:tensorflow:Loading model.


INFO:tensorflow:BLEURT initialized.


INFO:tensorflow:BLEURT initialized.


Extended version: 0.47095482013335105
The average score for the best matches: 0.5254725490793072
Concatenated version: 0.47439709543411074


## Huggingface interface
https://huggingface.co/spaces/evaluate-metric/bleurt/blob/main/README.md

In [15]:
bleurt_scorer = load("bleurt", module_type="metric", checkpoint="BLEURT-20")
references = ["This is a test.", "This is a test."]
candidates = ["This is a test.", "This is the test."]
scores = bleurt_scorer.compute(predictions=candidates, references=references)
print('Example:', np.mean(scores['scores']))
# Extended predictions
scores = bleurt_scorer.compute(predictions=memecap_predictions_extended, references=memecap_processed_gt_extended)
print('Extended version:', np.mean(scores['scores']))

# Concatenated references
scores = bleurt_scorer.compute(predictions=memecap_predictions_concatenated, references=memecap_processed_gt_concatenated)
print('Concatenated version:', np.mean(scores['scores']))

INFO:tensorflow:Reading checkpoint /home/shiling/.cache/huggingface/metrics/bleurt/default/downloads/extracted/13db1fc9d295585583f8a5538b232c82f61e73738f37827f3c4fc14396e60785/bleurt-base-128.


INFO:tensorflow:Reading checkpoint /home/shiling/.cache/huggingface/metrics/bleurt/default/downloads/extracted/13db1fc9d295585583f8a5538b232c82f61e73738f37827f3c4fc14396e60785/bleurt-base-128.


INFO:tensorflow:Config file found, reading.


INFO:tensorflow:Config file found, reading.


INFO:tensorflow:Will load checkpoint bert_custom


...

INFO:tensorflow:Loading model.


INFO:tensorflow:Loading model.


INFO:tensorflow:BLEURT initialized.


INFO:tensorflow:BLEURT initialized.


Example: 0.8682224154472351
Extended version: -0.6088926426224748
Concatenated version: -0.5713593499445967


# BERTscore

In [3]:
bertscore = load("bertscore")

In [6]:
# Extended predictions
results = bertscore.compute(predictions=memecap_predictions_extended, references=memecap_processed_gt_extended, lang="en")
print(results.keys())
print(np.mean(results['precision']), np.mean(results['recall']), np.mean(results['f1']))

calculate_the_best_matches(results['f1'], len(memecap_predictions_multi_ref), memecap_processed_gt_multi_ref)

# Concatenated references
results = bertscore.compute(predictions=memecap_predictions_concatenated, references=memecap_processed_gt_concatenated, lang="en")
print(np.mean(results['precision']), np.mean(results['recall']), np.mean(results['f1']))

# One-prediction to multi-reference
results = bertscore.compute(predictions=memecap_predictions_multi_ref, references=memecap_processed_gt_multi_ref, lang="en")
print(np.mean(results['precision']), np.mean(results['recall']), np.mean(results['f1']))

dict_keys(['precision', 'recall', 'f1', 'hashcode'])
0.8596986218782923 0.8784849302891948 0.8689020508858681
The average score for the best matches: 0.879493215859964
0.8675926138871702 0.8604510788773668 0.8639449427097008
0.8698499315771563 0.8925056580839485 0.879493215859964


# ChrF

## Huggingface interface

In [8]:
# prediction = ["The relationship between cats and dogs is not exactly friendly.", "a good bookshop is just a genteel black hole that knows how to read."]
# reference = [["The relationship between dogs and cats is not exactly friendly.", ], ["A good bookshop is just a genteel Black Hole that knows how to read."]]
chrf = load("chrf")

# {'score': 84.64214891738334, 'char_order': 6, 'word_order': 0, 'beta': 2}

# Extended predictions
results = chrf.compute(predictions=memecap_predictions_extended, references=memecap_processed_gt_extended)
print(results)
# Concatenated predictions
results = chrf.compute(predictions=memecap_predictions_concatenated, references=memecap_processed_gt_concatenated)
print(results)

In [11]:
# # one prediction to multiple references -- Huggingface
# # ValueError: ChrF, as implemented by sacrebleu, requires the same number of references for each prediction
# results = chrf.compute(predictions=memecap_predictions_multi_ref, references=memecap_processed_gt_multi_ref)
# print(results)

## Torchmetrics interface

In [12]:
# one prediction and multiple references
from torchmetrics.text import CHRFScore
# preds = ['the cat is on the mat']
# target = [['there is a cat on the mat', 'a cat is on the mat']]
chrf = CHRFScore()
chrf(memecap_predictions_multi_ref, memecap_processed_gt_multi_ref)

tensor(0.2883)

In [13]:
# Extended predictions
chrf(memecap_predictions_extended, memecap_processed_gt_extended)

tensor(0.2450)

In [14]:
# Concatenated references
chrf(memecap_predictions_concatenated, memecap_processed_gt_concatenated)

tensor(0.2077)

# ROUGE

In [18]:
rouge = load('rouge')
predictions = ["hello there", "general kenobi"]
references = ["hello there", "general kenobi"]

# {'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}

In [19]:
# Extended predictions
results = rouge.compute(predictions=memecap_predictions_extended,
                        references=memecap_processed_gt_extended)
print(results)

# Concatenated references
results = rouge.compute(predictions=memecap_predictions_concatenated,
                        references=memecap_processed_gt_concatenated)
print(results)

# one prediction VS multiple references
results = rouge.compute(predictions=memecap_predictions_multi_ref, 
                        references=memecap_processed_gt_multi_ref)
print(results)

{'rouge1': 0.21358079742963504, 'rouge2': 0.0416616902071557, 'rougeL': 0.17265236393343997, 'rougeLsum': 0.1726669393742032}


# bleu

In [22]:
# predictions = ["hello there general kenobi", "foo bar foobar"]
# references = [
#     ["hello there general kenobi", "hello there !"],
#     ["foo bar foobar"]
# ]
bleu =load("bleu")

# {'bleu': 1.0, 'precisions': [1.0, 1.0, 1.0, 1.0], 'brevity_penalty': 1.0, 'length_ratio': 1.1666666666666667, 'translation_length': 7, 'reference_length': 6}

# Extended predictions
bleu =load("bleu")
results = bleu.compute(predictions=memecap_predictions_extended, 
                       references=memecap_processed_gt_extended, max_order=4)
print(results)


# Concatenated references
results = bleu.compute(predictions=memecap_predictions_concatenated, 
                       references=memecap_processed_gt_concatenated)
print(results)

In [25]:
# one prediction VS multiple references
from nltk.tokenize import word_tokenize
results = bleu.compute(predictions=memecap_predictions_multi_ref, 
                       references=memecap_processed_gt_multi_ref, tokenizer=word_tokenize)
print(results)

{'bleu': 0.03673635280307039, 'precisions': [0.29231815091774305, 0.06801552459120698, 0.016127974824624663, 0.005679897220907431], 'brevity_penalty': 1.0, 'length_ratio': 2.7467323035138347, 'translation_length': 16181, 'reference_length': 5891}


# Justify that the sentence length has an influence on the final score for n-gram based metrics

In [2]:
predictions = [
    ["The meme poster is celebrating the husband's success in making his wife fall in love with him again after she lost her memory.",
    "The husband feels validated and proud that he was able to win his wife's heart for the second time.",
    "The meme poster is emphasizing the husband's worthiness and capability of rekindling the love with his wife despite the challenges." ],
    ["The meme poster is suggesting that The Simpsons once again predicted a real-life event involving Donald Trump and Greta Thunberg.",
    "Meme creator draws a parallel between Lisa Simpson's character and Greta Thunberg, emphasizing their roles as outspoken young activists.",
    "The poster is highlighting the uncanny resemblance between scenes from The Simpsons and actual events, suggesting the show has a history of accurately predicting the future."]
]

ground_truth = [[
    'Husband feels great after having their wife fall in love with him again after getting amnesia.',
    'The meme poster feels happy for the person who make his wife remember their love even after she forgot all.',
    'meme poster is conveying they feel like thor when a woman says to marrying them '], [
    "The Simpsons was correct about it's use of Trump and Greta Thurnberg. ",
    'The Simpsons is able to predict so many real life situation including Trump and Greta Thurnburg.'
    ]]
# concatnate all predictions and references of the same sample
memecap_predictions_fully_concatenated = []
memecap_processed_gt_fully_concatenated = []

# Flatten pridictions
new_predictions = []
new_gt = []
for idx, ite in enumerate(predictions):
    memecap_predictions_fully_concatenated.append(" ".join(ite))
    memecap_processed_gt_fully_concatenated.append(" ".join(ground_truth[idx]))
    for predict in ite:
        new_predictions.append(predict)
        new_gt.append(ground_truth[idx])
predictions = new_predictions
ground_truth = new_gt
len(predictions), len(ground_truth)

(6, 6)

In [3]:
# predictions = ["- The meme humorously suggests that a wife, after losing her memory, fell in love with and decided to remarry her own husband, indicating his unchanging worth and likeability as a partner, paralleled by the character's proud declaration of worthiness regarding his hammer.", 
#                'The meme humorously suggests that the TV show "The Simpsons" predicted real-life events involving Greta Thunberg and Donald Trump.']

# ground_truth = [[
#     'Husband feels great after having their wife fall in love with him again after getting amnesia.',
#     'The meme poster feels happy for the person who make his wife remember their love even after she forgot all.',
#     'meme poster is conveying they feel like thor when a woman says to marrying them '], [
#     "The Simpsons was correct about it's use of Trump and Greta Thurnberg. ",
#     'The Simpsons is able to predict so many real life situation including Trump and Greta Thurnburg.'
#     ]]

# len(predictions[0]), len(predictions[1])

In [3]:
gt = ground_truth

# concatenate all the references from the ground truth.
memecap_predictions_concatenated = []
memecap_processed_gt_concatenated = []
# copy the predictions to match the number of references
memecap_predictions_extended = []
memecap_processed_gt_extended = []
# Do nothing. One-prediction to multi-reference
memecap_predictions_multi_ref = predictions
memecap_processed_gt_multi_ref = gt

for idx, predic in enumerate(predictions):
    memecap_predictions_concatenated.append(predic)
    memecap_processed_gt_concatenated.append(''.join(gt[idx]))
    
    memecap_predictions_extended += [predic] * len(gt[idx])
    memecap_processed_gt_extended += gt[idx]

len(memecap_processed_gt_concatenated), len(memecap_processed_gt_extended), len(memecap_predictions_extended), len(memecap_predictions_multi_ref), len(memecap_processed_gt_multi_ref)

(6, 15, 15, 6, 6)

In [4]:
## BLEURT
from bleurt import score
checkpoint = "/home/shiling/git/bleurt/BLEURT-20"
bleurt_scorer = score.BleurtScorer(checkpoint)

# Extended predictions
scores = bleurt_scorer.score(references=memecap_processed_gt_extended, candidates=memecap_predictions_extended)
print('Extended version:', scores)
print('Mean of the extended version:', np.mean(scores))

# Calculate the score for the best matches 
restored_scores = []
last_idx = 0
for i in range(len(predictions)):
    restored_scores.append(scores[last_idx : last_idx + len(gt[i])])
    last_idx += len(gt[i])
print(restored_scores)
maximized = [ max(ite) for ite in restored_scores]
print('The average score for the best matches:', np.mean(maximized))

# Concatenated references
scores = bleurt_scorer.score(references=memecap_processed_gt_concatenated, candidates=memecap_predictions_concatenated)
print('Concatenated version:', scores)
print('Mean of the concatenated version:', np.mean(scores))

# Fully concatenated references
scores = bleurt_scorer.score(references=memecap_processed_gt_fully_concatenated, candidates=memecap_predictions_fully_concatenated)
print('Fully concatenated version:', scores)
print('Mean of the fully concatenated version:', np.mean(scores))

INFO:tensorflow:Reading checkpoint /home/shiling/git/bleurt/BLEURT-20.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint BLEURT-20
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:BLEURT-20
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... max_seq_length:512
INFO:tensorflow:... vocab_file:None
INFO:tensorflow:... do_lower_case:None
INFO:tensorflow:... sp_model:sent_piece
INFO:tensorflow:... dynamic_seq_length:True
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating SentencePiece tokenizer.
INFO:tensorflow:Creating SentencePiece tokenizer.
INFO:tensorflow:Will load model: /home/shiling/git/bleurt/BLEURT-20/sent_piece.model.
INFO:tensorflow:SentencePiece tokenizer created.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.


I0000 00:00:1722932062.079906    5332 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1722932062.080568    5332 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1722932062.080649    5332 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1722932062.081116    5332 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

INFO:tensorflow:BLEURT initialized.


INFO:tensorflow:BLEURT initialized.


Extended version: [0.5869571566581726, 0.6647629141807556, 0.4271870255470276, 0.5381008386611938, 0.5013132691383362, 0.3244766592979431, 0.4955701231956482, 0.5975973606109619, 0.4423678517341614, 0.536298394203186, 0.6327395439147949, 0.3608573079109192, 0.36088889837265015, 0.294441819190979, 0.38294482231140137]
Mean of the extended version: 0.47643359899520876
[[0.5869571566581726, 0.6647629141807556, 0.4271870255470276], [0.5381008386611938, 0.5013132691383362, 0.3244766592979431], [0.4955701231956482, 0.5975973606109619, 0.4423678517341614], [0.536298394203186, 0.6327395439147949], [0.3608573079109192, 0.36088889837265015], [0.294441819190979, 0.38294482231140137]]
The average score for the best matches: 0.529505729675293
Concatenated version: [0.5480329394340515, 0.48804110288619995, 0.5097494721412659, 0.6075407862663269, 0.41089391708374023, 0.5098664164543152]
Mean of the concatenated version: 0.5123541057109833
Fully concatenated version: [0.5299772620201111, 0.56549131870

In [18]:
np.mean([0.13559322033898305, 0.2777777777777778])

0.2066854990583804

In [5]:
## BERTscore
bertscore = load("bertscore")
# Extended predictions
results = bertscore.compute(predictions=memecap_predictions_extended, references=memecap_processed_gt_extended, lang="en")
print('Extended version:', np.mean(results['precision']), np.mean(results['recall']), np.mean(results['f1']))

# Concatenated references
results = bertscore.compute(predictions=memecap_predictions_concatenated, references=memecap_processed_gt_concatenated, lang="en")
print('Concatenated version:', np.mean(results['precision']), np.mean(results['recall']), np.mean(results['f1']))


# Fully concatenated references
results = bertscore.compute(predictions=memecap_predictions_fully_concatenated, references=memecap_processed_gt_fully_concatenated, lang="en")
print('Fully concatenated version:', np.mean(results['precision']), np.mean(results['recall']), np.mean(results['f1']))

# One-prediction to multi-reference
results = bertscore.compute(predictions=memecap_predictions_multi_ref, references=memecap_processed_gt_multi_ref, lang="en")
print('Multi-ref version:', np.mean(results['precision']), np.mean(results['recall']), np.mean(results['f1']))

/home/shiling/anaconda3/envs/llava/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Extended version: 0.8819923917452495 0.8825521032015483 0.8822169462839763
Concatenated version: 0.886967142422994 0.866187334060669 0.8764044344425201
Fully concatenated version: 0.8775137364864349 0.8948593735694885 0.8860620558261871
Multi-ref version: 0.8945286969343821 0.8968304892381033 0.8953786293665568


In [6]:
## CHrF
chrf = load("chrf")
# Extended predictions
results = chrf.compute(predictions=memecap_predictions_extended, references=memecap_processed_gt_extended)
print('Extended version:', results)
# Concatenated references
results = chrf.compute(predictions=memecap_predictions_concatenated, references=memecap_processed_gt_concatenated)
print('Concatenated version:', results)
# Fully concatenated references
results = chrf.compute(predictions=memecap_predictions_fully_concatenated, references=memecap_processed_gt_fully_concatenated)
print('Fully concatenated version:', results)

# ## CHrF
# chrf = load("chrf")
# print("sample 1:")
# # Extended predictions
# results = chrf.compute(predictions=memecap_predictions_extended[:2], references=memecap_processed_gt_extended[:2])
# print('Extended version:', results)
# # Concatenated references
# results = chrf.compute(predictions=[memecap_predictions_concatenated[0]], references=[memecap_processed_gt_concatenated[0]])
# print('Concatenated version:', results)

# print("sample 2:")
# # Extended predictions
# results = chrf.compute(predictions=memecap_predictions_extended[2:], references=memecap_processed_gt_extended[2:])
# print('Extended version:', results)
# # Concatenated references
# results = chrf.compute(predictions=[memecap_predictions_concatenated[1]], references=[memecap_processed_gt_concatenated[1]])
# print('Concatenated version:', results)

Extended version: {'score': 33.32482804288148, 'char_order': 6, 'word_order': 0, 'beta': 2}
Concatenated version: {'score': 25.869841671634354, 'char_order': 6, 'word_order': 0, 'beta': 2}
Fully concatenated version: {'score': 42.40475235853734, 'char_order': 6, 'word_order': 0, 'beta': 2}


In [7]:
## ROUGE
rouge = load('rouge')
print("sample 1:")
# Extended predictions
results = rouge.compute(predictions=memecap_predictions_extended,
                        references=memecap_processed_gt_extended)
print('Extended version:', results)

# Concatenated reference
results = rouge.compute(predictions=memecap_predictions_concatenated,
                        references=memecap_processed_gt_concatenated)
print('Concatenated version:', results)

# Fully concatenated reference
results = rouge.compute(predictions=memecap_predictions_fully_concatenated,
                        references=memecap_processed_gt_fully_concatenated)
print('Fully concatenated version:', results)

# One-prediction to multi-reference
results = rouge.compute(predictions=memecap_predictions_multi_ref,
                        references=memecap_processed_gt_multi_ref)
print('Multi-ref version:', results)

# ## ROUGE
# rouge = load('rouge')
# print("sample 1:")
# # Extended predictions
# results = rouge.compute(predictions=memecap_predictions_extended[:2],
#                         references=memecap_processed_gt_extended[:2])
# print('Extended version:', results)

# # Concatenated reference
# results = rouge.compute(predictions=[memecap_predictions_concatenated[0]],
#                         references=[memecap_processed_gt_concatenated[0]])
# print('Concatenated version:', results)

# # One-prediction to multi-reference
# results = rouge.compute(predictions=[memecap_predictions_multi_ref[0]],
#                         references=[memecap_processed_gt_multi_ref[0]])
# print('Multi-ref version:', results)

# print("sample 2:")
# # Extended predictions
# results = rouge.compute(predictions=memecap_predictions_extended[2:],
#                         references=memecap_processed_gt_extended[2:])
# print('Extended version:', results)

# # Concatenated reference
# results = rouge.compute(predictions=[memecap_predictions_concatenated[1]],
#                         references=[memecap_processed_gt_concatenated[1]])
# print('Concatenated version:', results)

# # One-prediction to multi-reference
# results = rouge.compute(predictions=[memecap_predictions_multi_ref[1]],
#                         references=[memecap_processed_gt_multi_ref[1]])
# print('Multi-ref version:', results)

sample 1:
Extended version: {'rouge1': 0.24159525176741817, 'rouge2': 0.11965887014686075, 'rougeL': 0.2192834451873189, 'rougeLsum': 0.22125348747299967}
Concatenated version: {'rouge1': 0.2717378924823593, 'rouge2': 0.1294849381423825, 'rougeL': 0.19014496793168825, 'rougeLsum': 0.19061430632859208}
Fully concatenated version: {'rouge1': 0.34410919540229884, 'rouge2': 0.1954087346024636, 'rougeL': 0.22126436781609193, 'rougeLsum': 0.22126436781609193}
Multi-ref version: {'rouge1': 0.3184737056688276, 'rouge2': 0.15808828287605878, 'rougeL': 0.27187631090070113, 'rougeLsum': 0.2736665306787258}


In [8]:
# BLEU
bleu =load("bleu")
print('Sample 1')
# Extended predictions
results = bleu.compute(predictions=memecap_predictions_extended, 
                       references=memecap_processed_gt_extended, max_order=4)
print('Extended version:', results)

# Concatenated reference
results = bleu.compute(predictions=memecap_predictions_concatenated, 
                       references=memecap_processed_gt_concatenated, max_order=4)
print('Concatenated version:', results)

# Fully concatenated reference
results = bleu.compute(predictions=memecap_predictions_fully_concatenated, 
                       references=memecap_processed_gt_fully_concatenated, max_order=4)
print('Fully concatenated version:', results)

# One-prediction to multi-reference
results = bleu.compute(predictions=memecap_predictions_multi_ref, 
                       references=memecap_processed_gt_multi_ref, max_order=4)
print('Multi-ref version:', results)

# ## BLEU
# bleu =load("bleu")
# print('Sample 1')
# # Extended predictions
# results = bleu.compute(predictions=memecap_predictions_extended[:2], 
#                        references=memecap_processed_gt_extended[:2], max_order=4)
# print('Extended version:', results)

# # Concatenated reference
# results = bleu.compute(predictions=[memecap_predictions_concatenated[0]], 
#                        references=[memecap_processed_gt_concatenated[0]], max_order=4)
# print('Concatenated version:', results)

# # One-prediction to multi-reference
# results = bleu.compute(predictions=[memecap_predictions_multi_ref[0]], 
#                        references=[memecap_processed_gt_multi_ref[0]], max_order=4)
# print('Multi-ref version:', results)

# print('Sample 2')
# # Extended predictions
# results = bleu.compute(predictions=memecap_predictions_extended[2:], 
#                        references=memecap_processed_gt_extended[2:], max_order=4)
# print('Extended version:', results)

# # Concatenated reference
# results = bleu.compute(predictions=[memecap_predictions_concatenated[1]], 
#                        references=[memecap_processed_gt_concatenated[1]], max_order=4)
# print('Concatenated version:', results)

# # One-prediction to multi-reference
# results = bleu.compute(predictions=[memecap_predictions_multi_ref[1]], 
#                        references=[memecap_processed_gt_multi_ref[1]], max_order=4)
# print('Multi-ref version:', results)

Sample 1
Extended version: {'bleu': 0.06142713874076677, 'precisions': [0.22388059701492538, 0.09375, 0.03934426229508197, 0.017241379310344827], 'brevity_penalty': 1.0, 'length_ratio': 1.3453815261044177, 'translation_length': 335, 'reference_length': 249}
Concatenated version: {'bleu': 0.054440737502768774, 'precisions': [0.37777777777777777, 0.17829457364341086, 0.08943089430894309, 0.042735042735042736], 'brevity_penalty': 0.4297960674895162, 'length_ratio': 0.5421686746987951, 'translation_length': 135, 'reference_length': 249}
Fully concatenated version: {'bleu': 0.11133493590639636, 'precisions': [0.2740740740740741, 0.15789473684210525, 0.07633587786259542, 0.046511627906976744], 'brevity_penalty': 1.0, 'length_ratio': 1.6265060240963856, 'translation_length': 135, 'reference_length': 83}
Multi-ref version: {'bleu': 0.1254059199425261, 'precisions': [0.362962962962963, 0.17829457364341086, 0.08943089430894309, 0.042735042735042736], 'brevity_penalty': 1.0, 'length_ratio': 1.607